In [ ]:
# Prepare DuckDB connection
import duckdb
conn = duckdb.connect()

In [ ]:
# Run your first query
conn.sql("SELECT 'mandarin' as duck")

In [ ]:
# Query public bucket on a Parquet file
conn.sql("FROM read_parquet('s3://us-prd-motherduck-open-datasets/hacker_news/parquet/hacker_news_2021_2022.parquet') limit 5")


In [ ]:
# Display install/loaded extensions
conn.sql("FROM duckdb_extensions();")

In [ ]:
# Install manually extensions
conn.sql("INSTALL spatial;")
conn.sql("LOAD spatial;")

In [ ]:
# Create a DuckDB tables based on a SELECT query
conn.sql("CREATE TABLE hacker_news_sample AS SELECT * FROM 's3://us-prd-motherduck-open-datasets/hacker_news/parquet/hacker_news_2021_2022.parquet' limit 5;")


In [ ]:
# Display all tables
conn.sql("SHOW ALL TABLES")

In [ ]:
# Use FROM first statement to directly query a table
conn.sql("FROM hacker_news_sample;")

In [ ]:
# Persist data by creating or attaching if exists a database
conn.sql("ATTACH 'my_hacker_news_stats.ddb';")
conn.sql("USE my_hacker_news_stats;")

# Create a larger table (10GB) in DuckDB
conn.sql("CREATE TABLE hacker_news_full AS SELECT * FROM 's3://us-prd-motherduck-open-datasets/hacker_news/parquet/hacker_news_2016_2025.zstd.parquet';")

# Display count
conn.sql("SELECT COUNT(*) from hacker_news_full;")

In [ ]:
conn.sql("ATTACH 'hacker_news_stats';")

# Compute the top domains shared
conn.sql("""CREATE TABLE hacker_news_stats.top_domains as (SELECT
    regexp_extract(url, 'http[s]?://([^/]+)/', 1) AS domain,
    count(*) AS count
FROM hacker_news_full
WHERE url IS NOT NULL AND regexp_extract(url, 'http[s]?://([^/]+)/', 1) != ''
GROUP BY domain
ORDER BY count DESC
LIMIT 20
);""")


In [ ]:
# Compute the mentions of DuckDB per month
conn.sql("""CREATE TABLE hacker_news_stats.duckdb_mentions AS (
SELECT
    YEAR(timestamp) AS year,
    MONTH(timestamp) AS month,
    COUNT(*) AS keyword_mentions
FROM hacker_news_full
WHERE
    (title LIKE '%duckdb%' OR text LIKE '%duckdb%')
GROUP BY year, month
ORDER BY year ASC, month ASC);""")

In [ ]:
# Export data to CSV using the COPY command
conn.sql("COPY (SELECT * FROM hacker_news_stats.top_domains) TO 'top_domains.csv'")

In [ ]:
# Create AWS secret based on sso chain (assuming you did `aws sso login before`)
conn.sql("""CREATE PERSISTENT SECRET aws_secret (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN
);""")


In [ ]:
# Display which secrets has been created
conn.sql("FROM duckdb_secrets();")

### Query with MotherDuck

In [ ]:
# Sign-up for free at motherduck.com and get your access token through the UI https://motherduck.com/docs/getting-started/sample-data-queries/hacker-news/
# Get motherduck_token stored in the notebook
from google.colab import userdata
import os
os.environ["motherduck_token"] = userdata.get('motherduck_token')

In [ ]:
# Connect to MotherDuck
conn.sql("ATTACH 'md:'")

In [ ]:
# Show your cloud databases
conn.sql('SHOW DATABASES')

In [ ]:
# Showing the query planner, explaining where things are being run (local/remote)
conn.sql("""EXPLAIN SELECT
    regexp_extract(url, 'http[s]?://([^/]+)/', 1) AS domain,
    count(*) AS count
FROM sample_data.hn.hacker_news
WHERE url IS NOT NULL AND regexp_extract(url, 'http[s]?://([^/]+)/', 1) != ''
GROUP BY domain
ORDER BY count DESC
LIMIT 20;""")

# Create a Cloud database
conn.sql("CREATE DATABASE cloud_hacker_news_stats;")

# Move a local DuckDB table to MotherDuck
conn.sql("CREATE TABLE cloud_hacker_news_stats.top_domains AS SELECT * FROM hacker_news_stats.top_domains;")

# Create a database share
conn.sql("CREATE SHARE my_share FROM cloud_hacker_news_stats;")

# Create a secret in MotherDuck
conn.sql("""CREATE SECRET aws_secret IN MOTHERDUCK (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN
);""")

In [ ]:
# Running this while connected to MotherDuck to leverage cloud compute & network bandwith
conn.sql("""SELECT
    regexp_extract(url, 'http[s]?://([^/]+)/', 1) AS domain,
    count(*) AS count
FROM 's3://us-prd-motherduck-open-datasets/hacker_news/parquet/hacker_news_2016_2025.zstd.parquet'
WHERE url IS NOT NULL AND regexp_extract(url, 'http[s]?://([^/]+)/', 1) != ''
GROUP BY domain
ORDER BY count DESC
LIMIT 20;""")